In [1]:
import matplotlib.pyplot as plt
import numpy as nm
import pandas as pd
import math
from rdkit import Chem
from rdkit.Chem import AllChem
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [2]:
folder = 'C:\\Users\\vswen\\Documents\\1. Biomedische Technologie\\BMT JAAR 5\\Kwart 4\\4. Data\\CTRPv2.0_2015_ctd2_ExpandedDataset\\'

# Import data
complete_df = pd.read_csv(f"{folder}scaffold_split.txt", sep="\t")
complete_df.fillna(complete_df.mean(), inplace=True)

C:\Users\vswen\AppData\Local\Temp\ipykernel_12708\3770761047.py:5: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  complete_df.fillna(complete_df.mean(), inplace=True)


In [3]:
# Fingerprint aanmaken
molecules = [Chem.MolFromSmiles(smile) for smile in complete_df['cpd_smiles'].tolist()]
ecfp = [AllChem.GetMorganFingerprintAsBitVect(molecule,2,nBits=1024) for molecule in molecules]
complete_df['ecfp_bit_vectors'] = [[int(bit) for bit in keys.ToBitString()] for keys in ecfp]
complete_df['ECFP'] = [''.join(str(value) for value in row) for row in complete_df['ecfp_bit_vectors']]


In [4]:
# Doelvariabele transformeren & limieten stellen dataframe
complete_df['ec50_mol'] = complete_df['apparent_ec50_umol'] / 1000000
complete_df['ec50_mol']=complete_df['ec50_mol'].replace(0, 1e-10)
complete_df['ec50_molair'] = complete_df['ec50_mol']/ complete_df['MolWt']
complete_df['ec50_molair_transformed'] = -nm.log10(complete_df['ec50_molair'])
condition = (complete_df['ec50_molair_transformed'] < 1 ) | (complete_df['ec50_molair_transformed'] > 10)
complete_df=complete_df[~condition]


In [5]:
# Dependent & Independent variable
x = nm.array(complete_df['ecfp_bit_vectors'].tolist())
y = complete_df['ec50_molair_transformed'].values
z = complete_df['recurring_scaffold'].values

In [55]:
x_train = []
y_train = []
x_test = []
y_test = []

0

In [49]:
total_length_compounds = len(complete_df)
size_train_set = total_length_compounds * 0.8
size_train_set_grouped_scaffolds = size_train_set * 0.8

259.2
The history saving thread hit an unexpected error (OperationalError('database or disk is full')).History will not be written to the database.


In [63]:
df_unique_scaffolds = complete_df[complete_df['recurring_scaffold'] == 0]
df_grouped_scaffolds = complete_df[complete_df['recurring_scaffold'] > 0]
df_grouped_scaffolds_indexes = df_grouped_scaffolds.groupby('recurring_scaffold')
#df_grouped_scaffolds['ecfp_bit_vectors'].get_group(2)

x=0
while len(x_train) < size_train_set_grouped_scaffolds:
        x_train.append(df_grouped_scaffolds['ecfp_bit_vectors'].get_group(x))
        y_train.append(df_grouped_scaffolds['ec50_molair_transformed'].get_group(x))
        x=x+1

    #print(df_grouped_scaffolds[['ecfp_bit_vectors','ec50_molair_transformed']].get_group(x))
#print(x_train)

AttributeError: 'Series' object has no attribute 'get_group'

In [32]:
x_grouped_scaffolds = df_grouped_scaffolds['cpd_smiles']
y_grouped_scaffolds = df_grouped_scaffolds['ec50_molair_transformed']


In [33]:
from sklearn.model_selection import GroupKFold

gkf_cv = GroupKFold(n_splits=2)

In [34]:
train_locations = pd.DataFrame(columns=['cpd_smiles', 'ec50_molair_transformed'])
test_locations = pd.DataFrame(columns=['cpd_smiles', 'ec50_molair_transformed'])

In [35]:
for split, (ix_train, ix_test) in enumerate(gkf_cv.split(x_grouped_scaffolds, y_grouped_scaffolds, groups=df_grouped_scaffolds['recurring_scaffold'])):
    print(f'SPLIT {split+1}')
    print(f'TRAIN INDEXES: {ix_train}, TEST INDEXES: {ix_test}\n')
    ix_train_list = nm.array(ix_train).tolist()
    print(ix_train_list)
    ix_test_list = nm.array(ix_test).tolist()
    print(ix_test_list)
    for i in ix_train_list:
        print (i)
       # train_locations=train_locations.append(pd.DataFrame({'cpd_smiles': x_grouped_scaffolds.iloc[i], 'ec50_molair_transformed': y_grouped_scaffolds.iloc[i]}),ignore_index=True)
    for j in ix_test_list:
        print (j)
       # test_locations=test_locations.append(pd.DataFrame({'cpd_smiles': x_grouped_scaffolds.iloc[j], 'ec50_molair_transformed': y_grouped_scaffolds.iloc[j]}),ignore_index=True)

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 2 dimensions. The detected shape was (23, 2) + inhomogeneous part.